# 3070 running hot

In [4]:
print("Hello world")

Hello world


In [5]:
#Check if cuda is available and running on the right gpu
import torch

torch.cuda.is_available()
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3070'

In [6]:
!python test_data_balanced.py


Summary per class:
B11: Copied 85, Missing 0
B12: Copied 85, Missing 0
B13: Copied 85, Missing 0
B14: Copied 85, Missing 0
B15: Copied 85, Missing 0
B16: Copied 85, Missing 0
B21: Copied 85, Missing 0
B22: Copied 85, Missing 0
B31: Copied 85, Missing 0
B32: Copied 85, Missing 0
B33: Copied 85, Missing 0
B55: Copied 85, Missing 0



Copying files: 100%|██████████| 1020/1020 [00:03<00:00, 270.42it/s]


In [7]:
!python train_data.py

Copied .\full_data_set\B12\LUCAS2006_34782616_Cover.jpg -> .\inputs/training\B12\LUCAS2006_34782616_Cover.jpg
Copied .\full_data_set\B13\LUCAS2006_34942610_Cover.jpg -> .\inputs/training\B13\LUCAS2006_34942610_Cover.jpg
Copied .\full_data_set\B15\LUCAS2006_33381766_Cover.jpg -> .\inputs/training\B15\LUCAS2006_33381766_Cover.jpg
Copied .\full_data_set\B21\LUCAS2006_32922910_Cover.jpg -> .\inputs/training\B21\LUCAS2006_32922910_Cover.jpg
Copied .\full_data_set\B32\LUCAS2006_35282642_Cover.jpg -> .\inputs/training\B32\LUCAS2006_35282642_Cover.jpg
Copied .\full_data_set\B21\LUCAS2009_44402394_Cover.jpg -> .\inputs/training\B21\LUCAS2009_44402394_Cover.jpg
Copied .\full_data_set\B15\LUCAS2009_45742784_Cover.jpg -> .\inputs/training\B15\LUCAS2009_45742784_Cover.jpg
Copied .\full_data_set\B12\LUCAS2009_47802066_Cover.jpg -> .\inputs/training\B12\LUCAS2009_47802066_Cover.jpg
Copied .\full_data_set\B33\LUCAS2009_40162700_Cover.jpg -> .\inputs/training\B33\LUCAS2009_40162700_Cover.jpg
Copied .\f

Preprocess the images for training, so that the Image Transformation does not have to be done for each training attempt.
The Transformation is pretty CPU and Disk I/O heavy.
Loading the preprocessed tensors eliminates that load. Also the tensors are kept in VRAM for training.

In [8]:
%run preprocess_transform.py --data_dir ./inputs/training --output_dir ./inputs/transformed_training
%run preprocess_transform.py --data_dir ./inputs/test_balanced --output_dir ./inputs/transformed_test_balanced

Preprocessing: 100%|██████████| 1020/1020 [00:35<00:00, 28.60it/s]


good batch_sizes would be denominators of 4800
100 92% GPU, 2.1G VRAM, 5s
200 90% GPU, 2.4G VRAM, 5s
300 93% GPU, 3.1G VRAM, 4s
400 93% GPU, 3.4G VRAM, 4s
600 93% GPU, 4.7G VRAM, 4s
800 93% GPU, 6.0G VRAM, 4s
1200 93% GPU, 7.7G VRAM, 5s

In [4]:
%load_ext autoreload
%autoreload 2
from train import main as main_train

main_train(
    data_dir_train='inputs/transformed_training', 
    data_dir_test='inputs/transformed_test_balanced',
    output_model_dir='outputs',
    batch_size=200,
    num_epochs=10,
    learning_rate=0.0035148759,
    resume=False,
    test=True
)

# %run train.py \
#     --data_dir_train inputs/transformed_training \
#     --batch_size 200 \
#     --num_epoch 10 \
#     --test True \
#     --data_dir_test ./inputs/transformed_test_balanced \
#     --output_model_dir ./outputs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cuda
Train Dataset: Found 4800 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']
Test Dataset: Found 1020 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']


Epoch 3/10:  79%|███████▉  | 19/24 [00:04<00:01,  4.19it/s]


KeyboardInterrupt: 

Run this in Terminal

```sh
uv run train.py
    --data_dir_train inputs/transformed_training
    --batch_size 200
    --num_epoch 3000
    --test True
    --data_dir_test inputs/transformed_test_balanced
    --output_model_dir outputs
```

`uv run tensorboard --logdir=runs`

# Running Vision Transformer

In [1]:
%load_ext autoreload
%autoreload 2
from train_vit_b_16 import main as main_train

main_train(
    data_dir_train='inputs/transformed_training', 
    data_dir_test='inputs/transformed_test_balanced',
    output_model_dir='outputs_vit',
    batch_size=100,
    num_epochs=10,
    learning_rate=0.0035148759,
    resume=False,
    test=True
)

c:\Users\johan\Ferris\farmers-eye-pytorch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Train Dataset: Found 4800 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']
Test Dataset: Found 1020 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']


c:\Users\johan\Ferris\farmers-eye-pytorch\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\johan\.cache\huggingface\hub\models--timm--vit_base_patch16_224.augreg2_in21k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Model to cuda
Criterion to cuda


Evaluating: 100%|██████████| 11/11 [00:16<00:00,  1.53s/it]


Test Accuracy=0.8461
Model saved to outputs_vit\000010_model.pth


In [ ]:
%load_ext autoreload
%autoreload 2
from train_vit_h_14 import main as main_train

main_train(
    data_dir_train='inputs/transformed_training', 
    data_dir_test='inputs/transformed_test_balanced',
    output_model_dir='outputs_vit',
    batch_size=200,
    num_epochs=10,  
    learning_rate=0.0035148759,
    resume=False,
    test=True
)

c:\Users\johan\Ferris\farmers-eye-pytorch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Train Dataset: Found 4800 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']
Test Dataset: Found 1020 images, 12 classes: ['B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B21', 'B22', 'B31', 'B32', 'B33', 'B55']
Model to cuda
Criterion to cuda


Evaluating: 100%|██████████| 6/6 [04:40<00:00, 46.76s/it]


Test Accuracy=0.5235
Model saved to outputs_vit\000010_model.pth


```sh

# Foundation Model